By default, the latest version of the API and the latest supported Spark version is chosen.
To specify your own: `%use spark(spark=3.3.0, scala=2.13, v=1.2.0)`

You can also define `displayLimit` and `displayTruncate` to control the display of the result.

Finally, any other property you pass, like `spark.master=local[4]`, will be passed on to Spark.

In [1]:
%use spark

received properties: Properties: {spark=3.3.1, scala=2.13, v=1.2.3, displayLimit=20, displayTruncate=30, spark.app.name=Jupyter, spark.master=local[*], spark.sql.codegen.wholeStage=false, fs.hdfs.impl=org.apache.hadoop.hdfs.DistributedFileSystem, fs.file.impl=org.apache.hadoop.fs.LocalFileSystem}, providing Spark with: {spark.app.name=Jupyter, spark.master=local[*], spark.sql.codegen.wholeStage=false, fs.hdfs.impl=org.apache.hadoop.hdfs.DistributedFileSystem, fs.file.impl=org.apache.hadoop.fs.LocalFileSystem}
23/09/02 10:21:31 INFO SparkContext: Running Spark version 3.3.1
23/09/02 10:21:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/09/02 10:21:32 INFO ResourceUtils: ==============================================================
23/09/02 10:21:32 INFO ResourceUtils: No custom resources configured for spark.driver.
23/09/02 10:21:32 INFO ResourceUtils: ===================================================

## establecemos los Datos de trabajo

Se define dos clases de enumeración: ColorOjos, Genero.
También se define una clase de datos: Persona

In [2]:
enum class ColorOjos {
    BLUE, BROWN, GREEN
}

enum class Genero {
    MALE, FEMALE, OTHER
}

In [3]:
data class Persona(
    val color_ojos: ColorOjos,
    val name: String,
    val gender: Genero,
    val length: Double,
    val age: Int,
)

Se define una variable "ds" que es un conjunto de datos de tipo Dataset<Persona>. El conjunto de datos contiene tres objetos "Persona", cada uno con diferentes propiedades, como color_ojos, name, gender, length (altura) y age.



In [4]:
val ds: Dataset<Persona> = dsOf(
    Persona(
        color_ojos = ColorOjos.BLUE,
        name = "Alice",
        gender = Genero.FEMALE,
        length = 1.70,
        age = 25,
    ),
    Persona(
        color_ojos = ColorOjos.BLUE,
        name = "Bob",
        gender = Genero.MALE,
        length = 1.67,
        age = 25,
    ),
    Persona(
        color_ojos = ColorOjos.BROWN,
        name = "Charlie",
        gender = Genero.OTHER,
        length = 1.80,
        age = 17,
    ),
)


In [5]:
// Se imprime "ds". La salida muestra los tres objetos Person.

ds

color_ojos,name,gender,length,age
BLUE,Alice,FEMALE,1.7,25
BLUE,Bob,MALE,1.67,25
BROWN,Charlie,OTHER,1.8,17


## Operaciones

Los efectos de operaciones como el filtrado también se pueden ver inmediatamente, así como la clasificación, selección de columnas, etc

In [6]:
ds.filter { it.age > 20 }

color_ojos,name,gender,length,age
BLUE,Alice,FEMALE,1.7,25
BLUE,Bob,MALE,1.67,25


In [7]:
ds.sort(col(Persona::age), col(Persona::length))

color_ojos,name,gender,length,age
BROWN,Charlie,OTHER,1.8,17
BLUE,Bob,MALE,1.67,25
BLUE,Alice,FEMALE,1.7,25


In [8]:
val res: Dataset<Tuple2<Int, Double>> = ds.select(col(Persona::age), col(Persona::length))
res

age,length
25,1.7
25,1.67
17,1.8


In [9]:
"Promedio de [length]: " +
    ds
        .map { it.length }
        .reduceK { a, b -> a + b } / ds.count()

Promedio de [length]: 1.7233333333333334

In [10]:
"Promedio de [age]: " +
    ds
        .map { it.age }
        .reduceK { a, b -> a + b } / ds.count()

Promedio de [age]: 22

También podemos crear RDD usando `sc: JavaSparkContext` que se representan de manera similar a los conjuntos de datos.
Puede ver que todas las funciones auxiliares de Tuple también están disponibles de inmediato.

In [11]:
val rdd: JavaRDD<Tuple2<Int, String>> = rddOf(
    1 X "aaa",
    t(2, "bbb"),
    tupleOf(3, "cccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccc"),
)

rdd

Values
"[1, aaa]"
"[2, bbb]"
"[3, ccccccccccccccccccccccc..."


In [12]:
val collected = rdd.collect() // Recopila todos los elementos en una lista

// Itera a través de la lista y muestra cada elemento
for (element in collected) {
    println(element)
}


(1,aaa)
(2,bbb)
(3,cccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccc)


In [13]:
val contar = rdd.filter { it._2.length > 3 }.count()
println("Número de tuplas con cadenas de longitud mayor a 3= $contar")


Número de tuplas con cadenas de longitud mayor a 3= 1


Finalmente, también podemos configurar diplayLimit y displayTruncate sobre la marcha usando sparkProperties.

In [14]:
sparkProperties {
    displayLimit = 2
    displayTruncate = -1
}

rdd

Values
"[1, aaa]"
"[2, bbb]"
